In [16]:
from google.colab import drive
drive.mount('/content/drive/')



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [17]:
import numpy as np
x = np.array([-1.5, -2, -1, 0.7, 2.3, -1.9])
W = np.array([[-3.5, 0, 3.1, 2.4, 1.8, 0.9], [1.7, -3.8, 0, 1.6, 2.3, -3.0], [2.8, 3.1, -2.9, 0, -2.1, -2.5]])
b = np.array([1.1, -2.1, -3.0])
W_t = np.transpose(W)
print(W_t)
y = np.dot(x,W_t) + b
print(y)

[[-3.5  1.7  2.8]
 [ 0.  -3.8  3.1]
 [ 3.1  0.  -2.9]
 [ 2.4  1.6  0. ]
 [ 1.8  2.3 -2.1]
 [ 0.9 -3.  -2.5]]
[  7.36  15.06 -10.58]


In [18]:
%autosave 60

Autosaving every 60 seconds


In [19]:
# from google.colab import files
# src = list(files.upload().values())[0]

import os
os.chdir("/content/drive/My Drive/CS444_assignments/CS444/assignment1")
import sys
# sys.path.append('/content/drive/My Drive/CS444_assignments/CS444/assignment1/')
sys.path.append(".")


In [20]:
!ls 
pwd = !pwd
print("Current working directory is: ",pwd)
!ls "models"

 Assign1_sandbox.ipynb	      ksa5_mp1_report.gdoc
 assignment1.zip	      ks-projects-201801-utf8.csv
 cifar_net.pth		      models
 colab_setup.ipynb	      mushroom
'CS 444 Assignment-1.ipynb'   mylib.py
 data			     'Numpy_logistic_reg_CS 444 Assignment-1.ipynb'
 data_process.py	      __pycache__
 fashion-mnist		      pytorch_tutorial.ipynb
 kaggle			      sandbox
 kaggle_submission.py	     'Sandbox_Assign1_CS 444.ipynb'
Current working directory is:  ['/content/drive/My Drive/CS444_assignments/CS444/assignment1']
__init__.py  logistic.py  perceptron.py  __pycache__  softmax.py  svm.py


In [21]:
import random
import numpy as np
import pandas as pd
# helpful character encoding module
import chardet
import math

from data_process import get_FASHION_data, get_MUSHROOM_data
from scipy.spatial import distance
# from models import Perceptron, SVM, Softmax, Logistic

from models.logistic import *
from models.perceptron import *
from models.softmax import *
from models.svm import *


from kaggle_submission import output_submission_csv
%matplotlib inline

# For auto-reloading external modules
# See http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
!pip install kaggle

# Loading Fashion-MNIST

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 60000]
, TEST_IMAGES = 10000

In [23]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 50000
VAL_IMAGES = 10000
normalize = True

In [24]:
!ls 

 Assign1_sandbox.ipynb	      ksa5_mp1_report.gdoc
 assignment1.zip	      ks-projects-201801-utf8.csv
 cifar_net.pth		      models
 colab_setup.ipynb	      mushroom
'CS 444 Assignment-1.ipynb'   mylib.py
 data			     'Numpy_logistic_reg_CS 444 Assignment-1.ipynb'
 data_process.py	      __pycache__
 fashion-mnist		      pytorch_tutorial.ipynb
 kaggle			      sandbox
 kaggle_submission.py	     'Sandbox_Assign1_CS 444.ipynb'


In [25]:
data = get_FASHION_data(TRAIN_IMAGES, VAL_IMAGES, normalize=normalize)
X_train_fashion, y_train_fashion = data['X_train'], data['y_train']
X_val_fashion, y_val_fashion = data['X_val'], data['y_val']
X_test_fashion, y_test_fashion = data['X_test'], data['y_test']
n_class_fashion = len(np.unique(y_test_fashion))

# Loading Mushroom


In the following cells we determine the splitting of the mushroom dataset.
<br /> TRAINING + VALIDATION = 0.8, TESTING = 0.2

In [26]:
# TRAINING = 0.6 indicates 60% of the data is used as the training dataset.
VALIDATION = 0.2

In [27]:
# TRAINING = 0.6 indicates 60% of the data is used as the training dataset.
VALIDATION = 0.2
data = get_MUSHROOM_data(VALIDATION)
X_train_MR, y_train_MR = data['X_train'], data['y_train']
X_val_MR, y_val_MR = data['X_val'], data['y_val']
X_test_MR, y_test_MR = data['X_test'], data['y_test']
n_class_MR = len(np.unique(y_test_MR))

print("Number of train samples: ", X_train_MR.shape[0])
print("Number of val samples: ", X_val_MR.shape[0])
print("Number of test samples: ", X_test_MR.shape[0])

Number of train samples:  4874
Number of val samples:  1625
Number of test samples:  1625


### Get Accuracy

This function computes how well your model performs using accuracy as a metric.

In [28]:
def get_acc(pred, y_test):
    return np.sum(y_test == pred) / len(y_test) * 100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
- **Learning rate** - controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. We recommend changing the learning rate by factors of 10 and observing how the performance of the classifier changes. You should also try adding a **decay** which slowly reduces the learning rate over each epoch.
- **Number of Epochs** - An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according to the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.

You will implement the Perceptron classifier in the **models/perceptron.py**

The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


## Train Perceptron on Fashion-MNIST

In [29]:
arr = np.array([1,2,3,7,12,768,2])
arr2 = np.arange(7)
#print(np.dot(np.transpose(arr),arr2))
weight = np.random.rand(2,2)
#print(arr.shape, weight.shape)
#print(weight)

#print(weight)

In [30]:
import numpy as np


class Perceptron:
    def __init__(self, n_class: int, lr: float, epochs: int):
        """Initialize a new classifier.

        Parameters:
            n_class: the number of classes
            lr: the learning rate
            epochs: the number of epochs to train for
        """
        self.w = None
        self.lr = lr
        self.epochs = epochs
        self.n_class = n_class

    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        """Train the classifier.

        Use the perceptron update rule as introduced in the Lecture.

        Parameters:
            X_train: a number array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        N, D = X_train.shape

        #self.w = np.random.rand(self.n_class,D)  # create a weight matrix of shape (1,D)
        self.w = np.zeros((self.n_class,D))
        #print(self.w)
        #print(self.w.shape)
        #print(y_train[0:20])
        for iter in range(self.epochs):
          #if iter > 5:
          #  self.lr = 0.5
          for example_num in range(N):
            x = X_train[example_num]
            y_label = y_train[example_num]
            y_hat_list = np.dot(self.w, x)  # get the dot product of weight and feature
            #print(y_label,y_hat_list)
            y_hat_max = np.argmax(y_hat_list)

            if y_label == y_hat_max:
              pass
            else:     # update weight
              y_yi = y_hat_list[y_label]      # correct label w^T_yi*xi
              #y_c = np.argwhere(y_hat_list > y_yi).reshape(1,-1)  # all labels higher than y_yi

              coef_x = (self.lr)*x

              for class_num in range(self.n_class):
                if iter == 0:
                  #if class_num == y_label:
                  self.w[y_label] = self.w[y_label] + coef_x
                  #else:
                  self.w[class_num] = self.w[class_num] - coef_x

                if y_hat_list[class_num] > y_yi:
                  self.w[y_label] = self.w[y_label] + coef_x
                  self.w[class_num] = self.w[class_num] - coef_x

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.

        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions

        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """
        N, D = X_test.shape
        labels = np.zeros((N))
        #print(self.w.shape)
        for example_num in range(N):
          x = X_test[example_num]
          y_hat = np.dot(self.w,x)
          labels[example_num] = np.argmax(y_hat)


        return labels

In [31]:
lr = 0.55
n_epochs = 10

percept_fashion = Perceptron(n_class_fashion, lr, n_epochs)
percept_fashion.train(X_train_fashion, y_train_fashion)

In [32]:
pred_percept = percept_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_fashion)))

The training accuracy is given by: 82.242000


### Validate Perceptron on Fashion-MNIST

In [33]:
pred_percept = percept_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_fashion)))

The validation accuracy is given by: 81.630000


### Test Perceptron on Fashion-MNIST

In [34]:
pred_percept = percept_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_fashion)))

The testing accuracy is given by: 80.790000


### Perceptron_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy, output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [35]:
#copy the kaggle.json token into kaggle folder
!mkdir -p ~/.kaggle
!cp kaggle/kaggle.json ~/.kaggle/

In [36]:
!pip install kaggle

In [37]:
!chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets list

In [38]:
#generate csv file for submission
output_submission_csv('kaggle/perceptron_submission_fashion.csv', percept_fashion.predict(X_test_fashion))

import pandas as pd
intermediate_dataframe = pd.read_csv("kaggle/perceptron_submission_fashion.csv")
intermediate_dataframe.to_csv('kaggle/perceptron_submission_fashion_utf8_encoding.csv', encoding='utf-8', index=False)


# # from https://www.kaggle.com/alexisbcook/character-encodings
# # look at the first ten thousand bytes to guess the character encoding
with open("kaggle/perceptron_submission_fashion_utf8_encoding.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)



# # from https://www.kaggle.com/alexisbcook/character-encodings
# # look at the first ten thousand bytes to guess the character encoding
# with open("kaggle/perceptron_submission_fashion.csv", 'rb') as rawdata:
#     result = chardet.detect(rawdata.read(10000))
# # check what the character encoding might be
# print(result)

# #check top lines
# intermediate_dataframe.head()


# # intermediate_dataframe.to_csv("kaggle/perceptron_submission_fashion_utf8_encoding.csv")







{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [39]:
# with open("kaggle/perceptron_submission_fashion.csv", 'rb') as source_file:
#   with open("kaggle/perceptron_submission_fashion_utf8_encoding.csv", 'w+b') as dest_file:
#     contents = source_file.read()
#     dest_file.write(contents.decode('utf-16-le').encode('utf-8'))

In [40]:
#measure the accuracy on the kaggle competition
# !kaggle competitions submit -c cs-444-assignment-1-perceptron -f kaggle/perceptron_submission_fashion_utf8_encoding.csv -m "Message"

## Train Perceptron on Mushroom

In [41]:
lr = 0.15
n_epochs = 10

percept_MR = Perceptron(n_class_MR, lr, n_epochs)
percept_MR.train(X_train_MR, y_train_MR)

In [45]:
pred_percept = percept_MR.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_MR)))

The training accuracy is given by: 94.521953


### Validate Perceptron on Mushroom

In [46]:
pred_percept = percept_MR.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_MR)))

The validation accuracy is given by: 94.030769


### Test Perceptron on Mushroom

In [47]:
pred_percept = percept_MR.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_MR)))

The testing accuracy is given by: 94.215385


# Support Vector Machines (with SGD)

Next, you will implement a "soft margin" SVM. In this formulation you will maximize the margin between positive and negative training examples and penalize margin violations using a hinge loss.

We will optimize the SVM loss using SGD. This means you must compute the loss function with respect to model weights. You will use this gradient to update the model weights.

SVM optimized with SGD has 3 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Epochs** - similar to as defined above in Perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case it is a coefficient on the term which maximizes the margin. You could try different values. The default value is set to 0.05.

You will implement the SVM using SGD in the **models/svm.py**

The following code: 
- Creates an instance of the SVM classifier class 
- The train function of the SVM class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train SVM on Fashion-MNIST

In [48]:
X = X_train_fashion
Y = y_train_fashion

N, D = X.shape
#print(shuffle_in_unison(X[0:100], Y[0:100]))
batch_size = 100
limit = N/batch_size
rand_num = np.random.randint(0,10)
slice_x = X[rand_num*batch_size:rand_num*batch_size+batch_size]
slice_y = Y[rand_num*batch_size:rand_num*batch_size+batch_size]
print(slice_x.shape, slice_y.shape)


(100, 784) (100,)


In [49]:
class SVM:
    def __init__(self, n_class: int, lr: float, epochs: int, reg_const: float,batch_size:int):
        """Initialize a new classifier.

        Parameters:
            n_class: the number of classes
            lr: the learning rate
            epochs: the number of epochs to train for
            reg_const: the regularization constant
        """
        self.w = None  # TODO: change this
        self.lr = lr
        self.epochs = epochs
        self.reg_const = reg_const
        self.n_class = n_class
        self.batch_size = batch_size
        self.learning_rate_exponent = learning_rate_exponent

    def calc_gradient(self, X_train: np.ndarray, y_train: np.ndarray) -> np.ndarray:
        """Calculate gradient of the svm hinge loss.

        Inputs have dimension D, there are C classes

        Parameters:
            X_train: a numpy array of shape (N, D) containing a mini-batch
                of data
            y_train: a numpy array of shape (N,) containing training labels;
                y[i] = c means that X[i] has label c, where 0 <= c < C

        Returns:
            the gradient with respect to weights w; an array of the same shape
                as w
        """
        x = X_train

        y_hat_list = self.reg_const + np.dot(self.reg_const + self.w, x)  # get the dot product of weight and feature

        return y_hat_list


    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        """Train the classifier.

        Hint: operate on mini-batches of data for SGD.

        Parameters:
            X_train: a numpy array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        N, D = X_train.shape
        batch_size = self.batch_size
        #self.w = np.random.uniform(low=0.1, high=0.8,size=(N,D))
        #self.w = np.zeros((N,D))
        self.w = np.random.rand(self.n_class,D)
        #print(self.w.shape)

        for iter in range(self.epochs):
          #if iter > 4:
          # self.lr -= iter*self.lr/9
          self.lr *= (self.learning_rate_exponent ** iter)
          # self.lr = self.lr * math.exp(-1*(self.learning_rate_exponent)*iter)
          print("lr: ",self.lr)

          #if self.lr > 6:
          # self.reg_const /= 0.9
          print("reg constant: ",self.reg_const)

          for example_num in range(0,N,batch_size):
            # print("example_num is: ",example_num)
            x = X_train[example_num]
            y_label = y_train[example_num]
            #print(y_label)
            #print(x.shape)
            y_hat_list = self.calc_gradient(x,y_label)
            y_correct = y_hat_list[y_label]
            #print(y_correct)
            #break
    
            for class_num in range(self.n_class):
              if y_correct != y_hat_list[class_num]:
                if y_correct - y_hat_list[class_num] < 1: 
                  self.w[y_label] = self.w[y_label] + self.lr*(x)
                  self.w[class_num] = self.w[class_num] - self.lr*(x)

              self.w[class_num] = (1 - self.lr*(self.reg_const/self.n_class))*self.w[class_num]
          print("weights are: ",self.w)
          print("Epoch number finished: ",iter)
        return

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.

        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions

        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """
        N, D = X_test.shape
        labels = np.zeros(N)

        for image_num in range(N):
          x = X_test[image_num]
          y_hat_list = np.dot(self.w, x)
          labels[image_num] = np.argmax(y_hat_list)
          if self.n_class == 2:
            labels[image_num] = np.where(labels[image_num] == -1, 0, labels[image_num])
 
        return labels

In [54]:
lr = 0.005
n_epochs = 10
reg_const = 0.3
learning_rate_exponent = 0.2
batch_size = 1

svm_fashion = SVM(n_class_fashion, lr, n_epochs, reg_const,batch_size)
svm_fashion.train(X_train_fashion, y_train_fashion)

lr:  0.005
reg constant:  0.3
weights are:  [[-6.24314173e-04 -5.55230464e-03 -9.25478590e-02 ... -4.03130853e+00
  -1.71707670e+00 -1.04273875e-01]
 [ 1.75761981e-04 -7.42850615e-03 -1.30564739e-01 ... -8.15411040e-01
  -1.73193354e-01 -9.03518050e-02]
 [-5.32913653e-04  2.59113932e-03 -1.26196347e-01 ...  2.86645575e+00
   6.66995134e-01  3.45573275e-01]
 ...
 [ 1.59836791e-04  7.00224201e-04  3.77266269e-03 ... -1.09449588e+00
  -4.20138668e-01 -1.63202080e-01]
 [-1.02328190e-03 -3.65098849e-03 -1.90665045e-01 ... -3.28371930e+00
  -1.65204458e+00 -2.50097619e-01]
 [ 2.19986439e-04 -1.50645528e-03  7.88603168e-04 ...  3.65801094e-01
   5.00197886e-01  1.38190374e-01]]
Epoch number finished:  0
lr:  0.001
reg constant:  0.3
weights are:  [[-2.11695709e-03  4.75492735e-02  1.50281286e-01 ... -1.47442160e+00
  -9.62468422e-01 -1.20471538e-01]
 [ 8.10993028e-05 -5.61367212e-03 -1.05337423e-01 ... -2.77032377e-01
   1.80112268e-01 -3.10966246e-02]
 [-4.10345612e-03 -1.39060549e-03 -3.685

In [55]:
pred_svm = svm_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_fashion)))

The training accuracy is given by: 84.134000


### Validate SVM on Fashion-MNIST

In [56]:
pred_svm = svm_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_fashion)))

The validation accuracy is given by: 82.730000


### Test SVM on Fashion-MNIST

In [57]:
pred_svm = svm_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_fashion)))

The testing accuracy is given by: 81.460000


### SVM_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [58]:
output_submission_csv('kaggle/svm_submission_fashion.csv', svm_fashion.predict(X_test_fashion))

## Train SVM on Mushroom

In [59]:
lr = 0.001
n_epochs = 10
reg_const = 0.6
batch_size = 1

svm_MR = SVM(n_class_MR, lr, n_epochs, reg_const,batch_size)
svm_MR.train(X_train_MR, y_train_MR)

lr:  0.001
reg constant:  0.6
weights are:  [[ 0.10878419 -0.03859232  0.11596207  0.16021186  0.1995664   0.16355074
   0.20828329 -0.05079235  0.30329904  0.26626704  0.25651566  0.25064784
   0.20805044  0.21310121  0.1895454   0.08497318 -0.08924052  0.20692438
   0.16721216  0.15434198 -0.15085448  0.08369037]
 [ 0.14191002  0.12491593  0.14407056  0.01724357  0.12072975  0.0771566
  -0.00427552  0.3604911   0.11069229 -0.00788924 -0.04946089 -0.05439747
   0.01167976  0.1627856   0.12778503  0.02280364  0.18341235  0.12714202
  -0.03623895  0.06669896  0.37081923  0.17711974]]
Epoch number finished:  0
lr:  0.0002
reg constant:  0.6
weights are:  [[ 0.07384644 -0.02093321  0.08087334  0.15628873  0.16445622  0.10791656
   0.18162225 -0.09496755  0.20829444  0.22261671  0.23560049  0.20941952
   0.17474792  0.14751032  0.13028818  0.06342683 -0.09359067  0.16010824
   0.14988877  0.12034391 -0.13692341  0.05231206]
 [ 0.11331193  0.08538558  0.1132677  -0.02382416  0.07466128  0.0

In [60]:
pred_svm = svm_MR.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_MR)))

The training accuracy is given by: 90.069758


### Validate SVM on Mushroom

In [61]:
pred_svm = svm_MR.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_MR)))

The validation accuracy is given by: 88.800000


## Test SVM on Mushroom

In [62]:
pred_svm = svm_MR.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_MR)))

The testing accuracy is given by: 88.800000


# Softmax Classifier (with SGD)

Next, you will train a Softmax classifier. This classifier consists of a linear function of the input data followed by a softmax function which outputs a vector of dimension C (number of classes) for each data point. Each entry of the softmax output vector corresponds to a confidence in one of the C classes, and like a probability distribution, the entries of the output vector sum to 1. We use a cross-entropy loss on this sotmax output to train the model. 

Check the following link as an additional resource on softmax classification: http://cs231n.github.io/linear-classify/#softmax

Once again we will train the classifier with SGD. This means you need to compute the gradients of the softmax cross-entropy loss function according to the weights and update the weights using this gradient. Check the following link to help with implementing the gradient updates: https://deepnotes.io/softmax-crossentropy

The softmax classifier has 3 hyperparameters that you can experiment with:
- **Learning rate** - As above, this controls how much the model weights are updated with respect to their gradient.
- **Number of Epochs** - As described for perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case, we minimize the L2 norm of the model weights as regularization, so the regularization constant is a coefficient on the L2 norm in the combined cross-entropy and regularization objective.

You will implement a softmax classifier using SGD in the **models/softmax.py**

The following code: 
- Creates an instance of the Softmax classifier class 
- The train function of the Softmax class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train Softmax on Fashion-MNIST

In [63]:
y = np.array(([-2.85],[0.86],[0.28]))
exp_y = np.exp(y)
log_k = -np.max(exp_y)
exp_y_logk = exp_y + log_k
#print(log_k)
sum_exp_y = np.sum(exp_y_logk)
#print(exp_y_logk)
#print(exp_y_logk/sum_exp_y)

In [64]:
z = np.random.uniform(low=0.01, high=0.1,size=(10,2))
#print(z)
#print(np.linalg.norm(z))

In [65]:
"""Softmax model."""

import numpy as np


class Softmax:
    def __init__(self, n_class: int, lr: float, epochs: int, reg_const: float):
        """Initialize a new classifier.

        Parameters:
            n_class: the number of classes
            lr: the learning rate
            epochs: the number of epochs to train for
            reg_const: the regularization constant
        """
        self.w = None  # TODO: change this
        self.lr = lr
        self.epochs = epochs
        self.reg_const = reg_const
        self.n_class = n_class

    def calc_gradient(self, X_train: np.ndarray, y_train: np.ndarray) -> np.ndarray:
        """Calculate gradient of the softmax loss.

        Inputs have dimension D, there are C classes, and we operate on
        mini-batches of N examples.

        Parameters:
            X_train: a numpy array of shape (N, D) containing a mini-batch
                of data
            y_train: a numpy array of shape (N,) containing training labels;
                y[i] = c means that X[i] has label c, where 0 <= c < C

        Returns:
            gradient with respect to weights w; an array of same shape as w
        """
        #N, D = X_train.shape
        #print(N,D)
        #gradients = np.zeros((N,D))
        x = X_train

        y_hat_list = np.dot(self.reg_const + self.w, x)  # get the dot product of weight and feature
        #print(y_hat_list)
        #exp_y = np.exp(y_hat_list)
        #print(exp_y)
        log_k = -np.max(y_hat_list)
        exp_y = np.exp(y_hat_list + log_k)
        sum_exp_y = np.sum(exp_y)
        gradients = exp_y / sum_exp_y

        return gradients

    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        """Train the classifier.

        Hint: operate on mini-batches of data for SGD.

        Parameters:
            X_train: a numpy array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        N, D = X_train.shape
        #self.w = np.random.uniform(low=0.1, high=0.8,size=(N,D))
        #self.w = np.zeros((N,D))
        self.w = np.random.rand(self.n_class,D)
        #print(self.w.shape)

        for iter in range(self.epochs):
          #if iter > 4:
          self.lr -= iter*self.lr/5

          #if self.lr > 6:
          self.reg_const /= 0.9

          for example_num in range(N):
            x = X_train[example_num]
            y_label = y_train[example_num]
            #print(y_label)
            #print(x.shape)
            gradients = self.calc_gradient(x,y_label)
            #print(gradients)
            #break
    
            for class_num in range(self.n_class):
              if class_num == y_label:
                self.w[y_label] = self.w[y_label] + (self.lr*(1 - gradients[y_label]))*x
              else:
                self.w[class_num] = self.w[class_num] - (self.lr*(gradients[class_num]))*x


        return

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.

        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions

        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """
        N, D = X_test.shape
        labels = np.zeros(N)

        for image_num in range(N):
          x = X_test[image_num]
          y_hat_list = np.dot(self.w, x)
          labels[image_num] = np.argmax(y_hat_list)
          if self.n_class == 2:
            labels[image_num] = np.where(labels[image_num] == -1, 0, labels[image_num])
 
        return labels

In [66]:
lr = 0.01
n_epochs = 14
reg_const = 0.55

softmax_fashion = Softmax(n_class_fashion, lr, n_epochs, reg_const)
softmax_fashion.train(X_train_fashion, y_train_fashion)

In [67]:
pred_softmax = softmax_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_fashion)))

The training accuracy is given by: 84.976000


### Validate Softmax on Fashion-MNIST

In [68]:
pred_softmax = softmax_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_fashion)))

The validation accuracy is given by: 81.580000


### Testing Softmax on Fashion-MNIST

In [69]:
pred_softmax = softmax_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_fashion)))

The testing accuracy is given by: 80.640000


### Softmax_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [70]:
output_submission_csv('kaggle/softmax_submission_fashion.csv', softmax_fashion.predict(X_test_fashion))

## Train Softmax on Mushroom

In [71]:
lr = 0.5
n_epochs = 10
reg_const = 0.05

softmax_MR = Softmax(n_class_MR, lr, n_epochs, reg_const)
#rint(n_class_MR)
softmax_MR.train(X_train_MR, y_train_MR)
print(y_train_MR.shape)

(4874,)


In [72]:
pred_softmax = softmax_MR.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_MR)))

The training accuracy is given by: 95.219532


### Validate Softmax on Mushroom

In [73]:
pred_softmax = softmax_MR.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_MR)))

The validation accuracy is given by: 94.523077


### Testing Softmax on Mushroom

In [74]:
pred_softmax = softmax_MR.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_MR)))

The testing accuracy is given by: 95.323077


# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.
- **Threshold** - The decision boundary of the classifier.


You will implement the Logistic Classifier in the **models/logistic.py**


The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

### Training Logistic Classifer

#Load mushroom dataset

In [75]:
# TRAINING = 0.6 indicates 60% of the data is used as the training dataset.
VALIDATION = 0.2
data = get_MUSHROOM_data(VALIDATION)
X_train_MR, y_train_MR = data['X_train'], data['y_train']
X_val_MR, y_val_MR = data['X_val'], data['y_val']
X_test_MR, y_test_MR = data['X_test'], data['y_test']
n_class_MR = len(np.unique(y_test_MR))

print("Number of train samples: ", X_train_MR.shape[0])
print("Number of val samples: ", X_val_MR.shape[0])
print("Number of test samples: ", X_test_MR.shape[0])

Number of train samples:  4874
Number of val samples:  1625
Number of test samples:  1625


In [76]:
# TAKE a look at the x_sub_i mushroom example values
#TOTAL examples: 4874
print("X_train_MR is: ", X_train_MR)

# there are 22 features = dimensions of each mushroom example
print("X_train_MR shape is: ", X_train_MR.shape)

#see the training label values
print("\ny_train_MR is: ", y_train_MR)

# see the shape of ytrain
print("y_train_MR shape is: ", y_train_MR.shape)

#################################################################
#see the validation label values
print("\ny_val_MR is: ", y_val_MR)

# see the shape of y for validation
print("y_val_MR is: ", y_val_MR.shape)
####################################################################


#see the testing label values
print("\ny_test_MR is: ", y_test_MR)

# see the shape of ytest
print("y_test_MR is: ", y_test_MR.shape)

# y_train_MR is:  [1 0 0 ... 1 1 0]
# y_train_MR is:  (4874,)
#we notice that y_train has 1,0 as labels. We need to replace the zero labels with -1.
#convert the zero in y to -1.
y_train_MR = np.array([-1 if value==0 else 1 for value in y_train_MR])
print("\n\n\nconverted_y_train is: ",y_train_MR)


#convert for y_val_MR as well
y_val_MR = np.array([-1 if value==0 else 1 for value in y_val_MR])
print("\nconverted_y_val_MR is: ",y_val_MR)

#convert for y_test_MR as well
y_test_MR = np.array([-1 if value==0 else 1 for value in y_test_MR])
print("\nconverted_y_test_MR is: ",y_test_MR)






X_train_MR is:  [[5 0 8 ... 3 4 0]
 [5 0 4 ... 2 3 1]
 [5 2 8 ... 3 3 3]
 ...
 [3 3 4 ... 7 4 4]
 [2 0 3 ... 1 5 4]
 [5 3 2 ... 7 1 6]]
X_train_MR shape is:  (4874, 22)

y_train_MR is:  [1 0 0 ... 1 1 0]
y_train_MR shape is:  (4874,)

y_val_MR is:  [1 0 0 ... 0 0 0]
y_val_MR is:  (1625,)

y_test_MR is:  [0 0 0 ... 0 0 0]
y_test_MR is:  (1625,)



converted_y_train is:  [ 1 -1 -1 ...  1  1 -1]

converted_y_val_MR is:  [ 1 -1 -1 ... -1 -1 -1]

converted_y_test_MR is:  [-1 -1 -1 ... -1 -1 -1]


In [77]:
def scalar_value_of_sigmoid(sigmoid_input):
        """Sigmoid function.

        Parameters:
            z: the input

        Returns:
            the sigmoid of the input
        """
        
        sigmoid_value = 1/(1+math.exp(-1*sigmoid_input))
        # print("sigmoid function returns: ",sigmoid_value)
        return sigmoid_value

In [78]:
# find the gradient of loss at a point
def sgd_gradient_of_loss_for_a_point(weight_vec,y_sub_i,x_sub_i,learning_rate,sigmoid_input,x_cols):


  # print("y_sub_i is: ",y_sub_i)

  sigmoid_input_for_gradient = -1*y_sub_i*(np.dot(x_sub_i,weight_vec))
  # print("sigmoid input of gradient is: ",sigmoid_input_for_gradient)
  # print("shape of sigmoid input of gradient is: ",sigmoid_input_for_gradient.shape)

  # print("x_sub_i is: ",x_sub_i)
  # print("x_sub_i shape is: ",x_sub_i.shape)

  # print("weight_vec is: ",weight_vec)
  # print("weight_vec shape is: ",weight_vec.shape)

  output_of_sigmoid_function = scalar_value_of_sigmoid(sigmoid_input_for_gradient)
  # print("output_of_sigmoid_function is: ",output_of_sigmoid_function)  


  gradient_of_loss_multiplied_by_eta = (x_sub_i)*learning_rate*(output_of_sigmoid_function)*y_sub_i
  # print("gradient_of_loss_multiplied_by_eta is: ",gradient_of_loss_multiplied_by_eta)
  # print("shape of gradient_of_loss_multiplied_by_eta is: ",gradient_of_loss_multiplied_by_eta.shape)
  gradient_of_loss_multiplied_by_eta = gradient_of_loss_multiplied_by_eta.reshape(x_cols,1)

  return gradient_of_loss_multiplied_by_eta


In [80]:
def get_acc(pred, y_test):
    return np.sum(y_test == pred) / len(y_test) * 100


In [82]:
"""Logistic regression model."""

import numpy as np


class Logistic:
    def __init__(self, lr: float, epochs: int, threshold: float):
        """Initialize a new classifier.

        Parameters:
            lr: the learning rate
            epochs: the number of epochs to train for
        """
        self.weight_vec = None  # TODO: change this
        self.lr = lr
        self.epoch_number = epochs
        self.threshold = threshold
        self.logistic_loss = []

    # def sigmoid(self, z: np.ndarray) -> np.ndarray:
    #     """Sigmoid function.

    #     Parameters:
    #         z: the input

    #     Returns:
    #         the sigmoid of the input
    #     """
    #     exp_z = np.exp(-z)
    #     # print("exp_z is: ",exp_z)
    #     # ones_array = np.ones(len(z))
    #     sum = 1+exp_z 
    #     print("sum is: ",sum)
    #     sigmoid_value = 1/(1+exp_z)
    #     print("sigmoid function returns: ",sigmoid_value)
    #     return sigmoid_value

    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        """Train the classifier.

        Use the *logistic regression update rule* as introduced in lecture.

        Parameters:
            X_train: a numpy array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        #in class notes, x_rows=n and x_cols=d
        x_rows,x_cols = X_train.shape
        self.weight_vec = np.zeros((x_cols,1))
        # print("self.weight_vec is: ",self.weight_vec)
        # print("self.weight_vec.shape is: ",self.weight_vec.shape)

        #reshape y_train to a column vector that is n by 1, 
        y_train = y_train.reshape(x_rows,1)
  

        #loop for each epoch
        for epoch_number in range(self.epoch_number):

            #we need to iterate over the weight matrix and take each row as input
            for x_row_index in range(x_rows):
                  x_row_for_example = X_train[x_row_index]
                  # print("x_row_for_example:",x_row_for_example)
                  y_label = y_train[x_row_index]
                  # print("y_label:",y_label)

                  sigmoid_input = y_label*np.dot(x_row_for_example,self.weight_vec)
                  sigmoid_output = scalar_value_of_sigmoid(sigmoid_input)
                  delta_weight_vector = sgd_gradient_of_loss_for_a_point(self.weight_vec,y_label,x_row_for_example,self.lr,sigmoid_input,x_cols)
                  # print("delta_weight_vector is: ",delta_weight_vector)
                  
                  # Updating the weight vector.
                  # print("weight vector before update is: ",self.weight_vec)
                  self.weight_vec = self.weight_vec + delta_weight_vector
                  # print("weight vector after update is: ",self.weight_vec)
              
            
        return self.weight_vec


        

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.

        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions

        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """

        N, D = X_test.shape
        labels = np.zeros((N))
        #print(self.w.shape)
        for example_num in range(N):
          x = X_test[example_num]
          y_hat = np.dot(x,self.weight_vec)
          if y_hat>=self.threshold:
            labels[example_num] = 1
          else:
            labels[example_num] = -1


        return labels


In [83]:
learning_rate = 0.6
n_epochs = 20
threshold = 0.5

lr = Logistic(learning_rate, n_epochs, threshold)
lr.train(X_train_MR, y_train_MR)

array([[  -2.01724437],
       [  22.23359795],
       [   1.65088044],
       [ -12.09568461],
       [ -14.47913435],
       [  42.75131934],
       [-178.24504802],
       [ 262.5225279 ],
       [  -5.72225053],
       [ -22.68513477],
       [ -68.93669088],
       [-147.59746428],
       [ -24.0227172 ],
       [  -2.93313362],
       [  -3.43449207],
       [   0.        ],
       [ 191.40812511],
       [   1.70174918],
       [  22.76219888],
       [ -22.57656537],
       [ -11.98547559],
       [   3.82978282]])

In [84]:
pred_lr = lr.predict(X_train_MR)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_MR)))

print("True y labels for training set of mushroom dataset are:",y_train_MR)
print("Predicted y labels for training set of mushroom dataset are:",pred_lr)


The training accuracy is given by: 94.870743
True y labels for training set of mushroom dataset are: [ 1 -1 -1 ...  1  1 -1]
Predicted y labels for training set of mushroom dataset are: [ 1. -1. -1. ...  1.  1. -1.]


### Validate Logistic Classifer

In [85]:
pred_lr = lr.predict(X_val_MR)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_MR)))

The validation accuracy is given by: 94.153846


### Test Logistic Classifier

In [86]:
pred_lr = lr.predict(X_test_MR)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_MR)))

The testing accuracy is given by: 94.461538
